In [ ]:
import numpy as np
import pandas as pd
import arcpy

In [ ]:
def table_to_df(layer):
    fields = []
    for field in arcpy.ListFields(layer):
        fields.append(field.name)
    arr = arcpy.da.TableToNumPyArray(layer, fields[2:])
    return pd.DataFrame(arr, columns=arr.dtype.names)

In [ ]:
df = table_to_df('CulturePlaceDC_duplicateIdentified')
df.head()

### groupby FEAT_SEQ for unique lat, lon and create a list of item names

In [ ]:
df.groupby('FEAT_SEQ')['Item'].apply(list)

In [ ]:
import os
import sys
sys.path.append(os.getcwd())

In [ ]:
from sgPlaceAPI import lat_lon_pk, lookup_pk_sgname

In [ ]:
csv_path = "./cultural_resource_dc.csv" 
df = pd.read_csv(csv_path)
df.head()

### A function remove unwanted characters (articles, comma, symbol) and leading and following whitespaces

In [ ]:
def removearticles(text):    
    stopwords = ['a', 'an', 'the', 'and', '&', ',', '.']
    textwords = text.split()
    resultwords  = [word for word in textwords if word.lower() not in stopwords]
    result = ' '.join(resultwords)
    return result

In [ ]:
df['Item'] = df['Item'].apply(removearticles)
df[:30]

In [ ]:
df['sg_pk'], df['sg_lat'], df['sg_lon'] = zip(*df.apply(lambda x: lat_lon_pk(x['Item'], x['lat'], x['lon']), axis=1))

In [ ]:
df.head()

In [ ]:
len(df)

In [ ]:
csv_path = "./cultural_resource_dc_sg.csv" 
df.to_csv(csv_path, index=False)